In [100]:
from pytorch_widedeep.preprocessing import WidePreprocessor, DeepPreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import BinaryAccuracy
import torch
from numpy import *

# these next 3 lines are not directly related to pytorch-widedeep. I assume
# you have downloaded the dataset and place it in a dir called data/adult/
# df = pd.read_csv('data/adult/adult.csv.zip')
# df['income_label'] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
# df.drop('income', axis=1, inplace=True)

# prepare wide, crossed, embedding and continuous columns
wide_cols  = [str(x) for x in list(range(22))]
#cross_cols = [list(str(y) for y in x) for x in [(0,4,5,11), (2,3,5,6,7,14,20)]]
cross_cols = []
already_dummies = [str(x) for x in list(range(4,22))]
embed_cols = [('0',16), ('1',16), ("2",16), ('3',32)]
cont_cols  = None
target_col = '22'

# target
target = df[target_col].values

# wide
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols, already_dummies=already_dummies)
# X_wide = preprocess_wide.fit_transform(df)
X_wide = X_wide[:,:-1]
wide = Wide(wide_dim=X_wide.shape[1], output_dim=1)

# deepdense
preprocess_deep = DeepPreprocessor(embed_cols=embed_cols, continuous_cols=cont_cols)
X_deep = preprocess_deep.fit_transform(df)
deepdense = DeepDense(hidden_layers=[64,32],
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input,
                      continuous_cols=cont_cols)

# build, compile, fit and predict
model = WideDeep(wide=wide, deepdense=deepdense)
optim = torch.optim.Adam(model.parameters())
model.compile(method='binary', metrics=[BinaryAccuracy], optimizers=optim)
model.fit(X_wide=X_wide, X_deep=X_deep, target=target, n_epochs=5, batch_size=256, val_split=0.2)

# X_wide_te = preprocess_wide.fit_transform(df_te)
X_deep_te = preprocess_deep.fit_transform(df_te)
model.predict(X_wide=X_wide_te[:,:-1], X_deep=X_deep_te)

  0%|          | 0/267 [00:00<?, ?it/s]

Training


predict: 100%|██████████| 58/58 [00:04<00:00, 13.02it/s]


array([1, 1, 0, ..., 0, 1, 1])

In [106]:
pred = model.predict(X_wide=X_wide_te[:,:-1], X_deep=X_deep_te)
label = X_wide_te[:,-1]
sum(pred==label)

predict: 100%|██████████| 58/58 [00:04<00:00, 12.41it/s]


8511

In [103]:
pred.shape

(14598,)

In [104]:
X_wide.shape
# u_content[1]
8511/14598

0.5830250719276613

In [25]:
from numpy import *
import pandas as pd

def data():
    path = "/home2/zh/data/ml-100k/u.data"
    with open(path, 'r', encoding='ISO-8859-1') as f:
        lines = f.read().split('\n')
    interaction = array([[int(x) for x in line.split('\t')] for line in lines[:-1]])
    #print(interaction[:3])
    interaction[:,:2] -= 1
    #print(interaction[:3])
    interaction[interaction[:,2]<=3, 2] = 0
    interaction[interaction[:,2]>3, 2] = 1
    #print(interaction[:3])

    def g(line):
        if line[2] == '':
            return [int(line[0])] + [0]*8 + line[-18:]
        return [int(line[0])] + list(eye(8)[int(line[2][-4:-1])-192]) + line[-18:]

    path = "/home2/zh/data/ml-100k/u.item"
    with open(path, 'r', encoding='ISO-8859-1') as f:
        lines = f.read().split('\n')
    #set([line.split('|')[2][-4:] for line in lines[:-1]])
    # line = lines[23].split('|')
    # array([int(line[0])] + list(eye(8)[int(line[2][-4:-1])-192]) + line[-18:]).astype(int)
    i_content = array([g(line.split('|')) for line in lines[:-1]]).astype(float)[:,1:]

    def h(line):
        #bl = [20, 30, 40, 50]
        a = [0]*28
        b = int(line[1])
        if b <= 20: a[0] = 1
        elif b <= 30: a[1] = 1
        elif b <= 40: a[2] = 1
        elif b <= 50: a[3] = 1
        else: a[4] = 1
        if line[2] == 'F': a[5] = 1
        else: a[6] = 1
        a[dict(zip(['administrator',
            'artist',
            'doctor',
            'educator',
            'engineer',
            'entertainment',
            'executive',
            'healthcare',
            'homemaker',
            'lawyer',
            'librarian',
            'marketing',
            'none',
            'other',
            'programmer',
            'retired',
            'salesman',
            'scientist',
            'student',
            'technician',
            'writer'], range(7,28)))[line[3]]] = 1
        return a

    path = "/home2/zh/data/ml-100k/u.user"
    with open(path, 'r', encoding='ISO-8859-1') as f:
        lines = f.read().split('\n')
    #set([line.split('|')[3] for line in lines[:-1]])
    u_content = array([h(line.split('|')) for line in lines[:-1]])
    #sum(i_content[:,0]==1)
    
    return interaction, u_content, i_content

In [98]:
interaction, u_content, i_content = data()
perm = random.permutation(943)
df, df_te = [], []
X_wide, X_wide_te = [], []
rev_u = {'age': (['20-', '20s', '30s', '40s', '50+'], 0, 5),
         'gender': (['F', 'M'], 5, 7),
         'occupation': (['administrator', 'artist', 'doctor', 'educator', 'engineer',
                        'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
                        'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
                        'salesman', 'scientist', 'student', 'technician', 'writer'], 7, 28)}
rev_i = {'decade': ([str(x)+'0s' for x in range(192,200)], 0, 8)}
for u, i, r, t in interaction:
    a = [rev_u[attname][0][argmax(u_content[u][rev_u[attname][1]:rev_u[attname][2]])] for attname in rev_u.keys()]
    b = [rev_i[attname][0][argmax(i_content[i][rev_i[attname][1]:rev_i[attname][2]])] for attname in rev_i.keys()]
    c = hstack((u_content[u], i_content[i], r))
#     print(c.shape)
#     break
    if u in perm[:800]:
        df.append(a+b+list(i_content[i][8:])+[r])
        X_wide.append(c)
    else:
        df_te.append(a+b+list(i_content[i][8:])+[r])
        X_wide_te.append(c)
df = pd.DataFrame(df, columns=[str(x) for x in range(23)])#, columns=['age', 'gender', 'occupation', 'decade'])
df_te = pd.DataFrame(df_te, columns=[str(x) for x in range(23)])
X_wide = vstack(X_wide)
X_wide_te = vstack(X_wide_te)